In [6]:
! pip install --q unstructured langchain
# ! pip install --"unstructured[all-docs]"

# Ingesting PDF

In [1]:
from pprint import pprint
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain_community.document_loaders import PyPDFLoader

In [4]:
local_path = "/mnt/c/Users/Green.PC/Documents/ASAMA/project/rag-chatbot-telegram-data/data/WEF_The_Global_Cooperation_Barometer_2024.pdf"

if local_path:
    loader = PyPDFLoader(file_path=local_path)
    data = loader.load_and_split()
else:
    print('Upload a PDF file')

In [5]:
pprint(data[0].page_content)

('The Global Cooperation \n'
 'Barometer 2024\n'
 'INSIGHT REPORT\n'
 'JANUARY 2024In collaboration with \n'
 'McKinsey & Company')


# Vector Embeddings

**"ollama serve" in terminal**

In [10]:
# !docker exec -it ollama ollama pull nomic-embed-text
!docker exec -it ollama ollama list
!ollama pull nomic-embed-text
!ollama list

Error response from daemon: No such container: ollama
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% 

In [3]:
!pip install --q chromadb
!pip install --q langchain-text-splitters

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [ ]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name='local-rag'
    
)

# Retrieval

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question:{question}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input(""))

In [ ]:
chain.invoke("What are the 5 pillars of global cooperation?")